In [ ]:
import utils

In [1]:
import os
os.chdir('../../..')

In [2]:
import convokit

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from convokit import Corpus, download
from convokit import HyperConvo
import numpy as np
import pickle

In [4]:
corpus = Corpus(filename=download('reddit-corpus-small'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/reddit-corpus-small


## 1. First, generate the 3-way tensor (hypergraph_features x (time within comment thread) x threads in subreddit)

Let's get a sense of how we should normalize the time within threads.

In [5]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [6]:
corpus = corpus.reindex_conversations(top_level_comment_ids)

In [7]:
corpus.print_summary_stats()

Number of Users: 116643
Number of Utterances: 288846
Number of Conversations: 10000


In [9]:
hc_transformers = [HyperConvo(prefix_len=i, feat_name="hyperconvo-{}".format(i)) for i in range(2, 11)]

In [10]:
len(hc_transformers)

10

In [ ]:
for idx, hc in enumerate(list(reversed(hc_transformers))):
    print(10-idx)
    hc.transform(corpus)

Now we construct the tensor:

In [76]:
tensor = np.zeros((9, 10000, 164))

In [77]:
tensor.shape

(9, 10000, 164)

In [78]:
for convo_idx, convo in enumerate(corpus.iter_conversations()):
    for hyperconvo_idx in range(2, 10+1):
        tensor[hyperconvo_idx-2][convo_idx] = list(convo.meta['hyperconvo-{}'.format(hyperconvo_idx)].values())

In [84]:
# impute NaN = -1
tensor[np.isnan(tensor)] = -1

Save the tensor for future use:

In [90]:
with open('convokit/tensor/hypergraph_tensor.p', 'wb') as f:
    pickle.dump(tensor, f)

In [92]:
from tensorly.decomposition import parafac

In [94]:
rank = 3
factors_tl = parafac(tensor, rank=rank)

In [98]:
weights_factors = factors_tl

In [99]:
factors = weights_factors[1]

In [100]:
factors

[array([[  -66.40266874,   221.2209921 ,   -33.25618556],
        [ -112.39940896,   456.97747934,   103.78562753],
        [ -379.17544745,   565.31842325,   169.16896242],
        [ -792.41028346,   663.88399972,   223.14641012],
        [-1342.35278634,   762.76973325,   272.0521014 ],
        [-2028.34359701,   860.8941223 ,   319.38382641],
        [-2825.74506353,   963.28307538,   367.69721017],
        [-3722.59722203,  1068.50741587,   417.6908409 ],
        [-4704.86557798,  1178.87180617,   470.50573684]]),
 array([[-0.00367843, -0.01095659, -0.01271528],
        [-0.00384389, -0.01285473, -0.01186959],
        [-0.00079181, -0.01775238, -0.01374339],
        ...,
        [-0.0086283 , -0.01188227, -0.01217201],
        [ 0.00038416, -0.01615329, -0.01276264],
        [-0.000353  , -0.01822856, -0.01159199]]),
 array([[ 1.73443619e-03, -1.66662650e-01,  1.91967695e-01],
        [-2.04671934e-02, -3.95131698e-01,  2.59905677e-01],
        [ 1.73443619e-03, -1.66662650e-01,  1

In [105]:
os.chdir('convokit/tensors')

In [111]:
from utils import *

In [112]:
import utils

In [113]:
utils

<module 'utils' from '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/utils/__init__.py'>

In [114]:
os.listdir()

['demo', '__init__.py', '__pycache__', 'utils.py', 'data']

In [109]:
plot_afactors(factors)

NameError: name 'plot_factors' is not defined